In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Lasso, Ridge, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# ファイルパスの設定
file_path = r"G:\共有ドライブ\GAP_長寿研\user\iwamoto\視線の動きの俊敏さ\data\509k1.csv"

# 出力ディレクトリの設定
output_dir = r"G:\共有ドライブ\GAP_長寿研\user\iwamoto\視線の動きの俊敏さ\smoteresult"

# 出力ディレクトリが存在しない場合は作成
os.makedirs(output_dir, exist_ok=True)

class FeatureSelectionAnalyzer:
    def __init__(self, output_dir=None):
        self.scaler = StandardScaler()
        self.lasso_model = None
        self.ridge_model = None
        self.selected_features = None
        self.output_dir = output_dir or "results"
        
        # 出力ディレクトリが存在しない場合は作成
        os.makedirs(self.output_dir, exist_ok=True)
        
    def load_data(self, file_path):
        """データの読み込み"""
        try:
            # CSVファイルの読み込み
            self.data = pd.read_csv(file_path)
            print(f"データサイズ: {self.data.shape}")
            print(f"列名: {list(self.data.columns)}")
            return True
        except Exception as e:
            print(f"データ読み込みエラー: {e}")
            return False
    
    def prepare_data(self, target_column='target'):
        """データの前処理"""
        # 目的変数の設定
        if target_column not in self.data.columns:
            print(f"目的変数 '{target_column}' が見つかりません")
            return False
            
        # 特徴量と目的変数を分離
        X = self.data.drop([target_column, 'InspectionDateAndId'], axis=1, errors='ignore')
        y = self.data[target_column]
        
        # 欠損値の処理
        X = X.fillna(X.mean())
        y = y.fillna(y.mean())
        
        # 訓練・テストデータの分割
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # 特徴量の標準化
        self.X_train_scaled = self.scaler.fit_transform(self.X_train)
        self.X_test_scaled = self.scaler.transform(self.X_test)
        
        # 特徴量名を保存
        self.feature_names = X.columns.tolist()
        
        print(f"特徴量数: {len(self.feature_names)}")
        print(f"訓練データ: {self.X_train.shape}, テストデータ: {self.X_test.shape}")
        
        return True
    
    def lasso_feature_selection(self, alpha_range=None):
        """LASSO回帰による特徴量選択"""
        if alpha_range is None:
            alpha_range = np.logspace(-4, 1, 50)
        
        # クロスバリデーションでαを最適化
        lasso_cv = LassoCV(alphas=alpha_range, cv=5, random_state=42, max_iter=10000)
        lasso_cv.fit(self.X_train_scaled, self.y_train)
        
        # 最適なαでLASSO回帰を実行
        self.lasso_model = Lasso(alpha=lasso_cv.alpha_, max_iter=10000)
        self.lasso_model.fit(self.X_train_scaled, self.y_train)
        
        # 選択された特徴量を取得
        lasso_coef = self.lasso_model.coef_
        selected_idx = np.where(np.abs(lasso_coef) > 1e-6)[0]
        
        self.selected_features = {
            'indices': selected_idx,
            'names': [self.feature_names[i] for i in selected_idx],
            'coefficients': lasso_coef[selected_idx]
        }
        
        print(f"最適なα (LASSO): {lasso_cv.alpha_:.6f}")
        print(f"選択された特徴量数: {len(selected_idx)} / {len(self.feature_names)}")
        print(f"削減率: {(1 - len(selected_idx) / len(self.feature_names)) * 100:.1f}%")
        
        return lasso_cv.alpha_, selected_idx
    
    def ridge_regression(self, alpha_range=None):
        """Ridge回帰の実行"""
        if alpha_range is None:
            alpha_range = np.logspace(-2, 2, 50)
        
        # クロスバリデーションでαを最適化
        ridge_cv = RidgeCV(alphas=alpha_range, cv=5)
        ridge_cv.fit(self.X_train_scaled, self.y_train)
        
        # 最適なαでRidge回帰を実行
        self.ridge_model = Ridge(alpha=ridge_cv.alpha_)
        self.ridge_model.fit(self.X_train_scaled, self.y_train)
        
        print(f"最適なα (Ridge): {ridge_cv.alpha_:.6f}")
        
        return ridge_cv.alpha_
    
    def evaluate_models(self):
        """モデルの評価"""
        results = {}
        
        if self.lasso_model is not None:
            # LASSO回帰の評価
            lasso_pred_train = self.lasso_model.predict(self.X_train_scaled)
            lasso_pred_test = self.lasso_model.predict(self.X_test_scaled)
            
            results['LASSO'] = {
                'train_r2': r2_score(self.y_train, lasso_pred_train),
                'test_r2': r2_score(self.y_test, lasso_pred_test),
                'train_mse': mean_squared_error(self.y_train, lasso_pred_train),
                'test_mse': mean_squared_error(self.y_test, lasso_pred_test)
            }
        
        if self.ridge_model is not None:
            # Ridge回帰の評価
            ridge_pred_train = self.ridge_model.predict(self.X_train_scaled)
            ridge_pred_test = self.ridge_model.predict(self.X_test_scaled)
            
            results['Ridge'] = {
                'train_r2': r2_score(self.y_train, ridge_pred_train),
                'test_r2': r2_score(self.y_test, ridge_pred_test),
                'train_mse': mean_squared_error(self.y_train, ridge_pred_train),
                'test_mse': mean_squared_error(self.y_test, ridge_pred_test)
            }
        
        return results
    
    def plot_coefficient_comparison(self, top_n=20):
        """係数の比較プロット"""
        if self.lasso_model is None or self.ridge_model is None:
            print("モデルが訓練されていません")
            return
        
        # 係数を取得
        lasso_coef = self.lasso_model.coef_
        ridge_coef = self.ridge_model.coef_
        
        # 絶対値の大きい順に特徴量をソート
        coef_abs = np.abs(lasso_coef) + np.abs(ridge_coef)
        top_indices = np.argsort(coef_abs)[-top_n:][::-1]
        
        # プロット用データの準備
        plot_data = pd.DataFrame({
            'Feature': [self.feature_names[i] for i in top_indices],
            'LASSO': lasso_coef[top_indices],
            'Ridge': ridge_coef[top_indices]
        })
        
        # プロット
        plt.figure(figsize=(12, 8))
        x = np.arange(len(plot_data))
        width = 0.35
        
        plt.bar(x - width/2, plot_data['LASSO'], width, label='LASSO', alpha=0.8)
        plt.bar(x + width/2, plot_data['Ridge'], width, label='Ridge', alpha=0.8)
        
        plt.xlabel('特徴量')
        plt.ylabel('係数')
        plt.title('LASSO vs Ridge 係数比較')
        plt.xticks(x, plot_data['Feature'], rotation=45, ha='right')
        plt.legend()
        plt.tight_layout()
        
        # 図を保存
        save_path = os.path.join(self.output_dir, 'coefficient_comparison.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        print(f"図を保存しました: {save_path}")
        
        # データも保存
        csv_path = os.path.join(self.output_dir, 'coefficient_comparison.csv')
        plot_data.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"データを保存しました: {csv_path}")
    
    def plot_feature_selection_path(self, alpha_range=None):
        """特徴量選択パスの可視化"""
        if alpha_range is None:
            alpha_range = np.logspace(-4, 1, 50)
        
        # 各αに対するLASSO係数を計算
        coef_path = []
        for alpha in alpha_range:
            lasso = Lasso(alpha=alpha, max_iter=10000)
            lasso.fit(self.X_train_scaled, self.y_train)
            coef_path.append(lasso.coef_)
        
        coef_path = np.array(coef_path)
        
        # プロット（重要な特徴量のみ）
        plt.figure(figsize=(12, 8))
        
        # 最終的に残る特徴量を特定
        final_nonzero = np.where(np.abs(coef_path[-1]) > 1e-6)[0]
        
        for i in final_nonzero[:10]:  # 上位10個のみプロット
            plt.plot(alpha_range, coef_path[:, i], 
                    label=f'{self.feature_names[i][:15]}...', linewidth=2)
        
        plt.xscale('log')
        plt.xlabel('正則化パラメータ α')
        plt.ylabel('係数')
        plt.title('LASSO 特徴量選択パス')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        # 図を保存
        save_path = os.path.join(self.output_dir, 'lasso_path.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        print(f"図を保存しました: {save_path}")
    
    def get_selected_features_info(self):
        """選択された特徴量の情報を取得"""
        if self.selected_features is None:
            print("特徴量選択が実行されていません")
            return None
        
        # 係数の絶対値でソート
        sorted_indices = np.argsort(np.abs(self.selected_features['coefficients']))[::-1]
        
        results = []
        for i in sorted_indices:
            idx = self.selected_features['indices'][i]
            results.append({
                'feature_name': self.selected_features['names'][i],
                'coefficient': self.selected_features['coefficients'][i],
                'abs_coefficient': np.abs(self.selected_features['coefficients'][i])
            })
        
        df_results = pd.DataFrame(results)
        
        # 結果を保存
        save_path = os.path.join(self.output_dir, 'selected_features.csv')
        df_results.to_csv(save_path, index=False, encoding='utf-8-sig')
        print(f"選択された特徴量を保存しました: {save_path}")
        
        return df_results
    
    def save_results_summary(self, results):
        """結果サマリーの保存"""
        summary = []
        
        if 'LASSO' in results:
            summary.append({
                'Model': 'LASSO',
                'Train_R2': results['LASSO']['train_r2'],
                'Test_R2': results['LASSO']['test_r2'],
                'Train_MSE': results['LASSO']['train_mse'],
                'Test_MSE': results['LASSO']['test_mse']
            })
        
        if 'Ridge' in results:
            summary.append({
                'Model': 'Ridge',
                'Train_R2': results['Ridge']['train_r2'],
                'Test_R2': results['Ridge']['test_r2'],
                'Train_MSE': results['Ridge']['train_mse'],
                'Test_MSE': results['Ridge']['test_mse']
            })
        
        df_summary = pd.DataFrame(summary)
        save_path = os.path.join(self.output_dir, 'model_evaluation_summary.csv')
        df_summary.to_csv(save_path, index=False, encoding='utf-8-sig')
        print(f"モデル評価サマリーを保存しました: {save_path}")
        
        return df_summary

# 使用例
def main():
    # 分析器の初期化（出力ディレクトリを指定）
    analyzer = FeatureSelectionAnalyzer(output_dir=output_dir)
    
    # データの読み込み
    print(f"データファイル: {file_path}")
    print(f"出力ディレクトリ: {output_dir}")
    
    if analyzer.load_data(file_path):
        
        # データの前処理（目的変数を適切に設定してください）
        # 注意: 'target'列が存在しない場合は、適切な列名に変更してください
        target_candidates = ['target', 'Age', 'prediction', 'actual']  # 可能性のある目的変数名
        target_column = None
        
        for candidate in target_candidates:
            if candidate in analyzer.data.columns:
                target_column = candidate
                break
        
        if target_column is None:
            print("利用可能な列:")
            print(analyzer.data.columns.tolist())
            print("適切な目的変数を指定してください")
            return
        
        print(f"目的変数として '{target_column}' を使用します")
        
        if analyzer.prepare_data(target_column=target_column):
            
            # LASSO回帰による特徴量選択
            print("\n=== LASSO回帰による特徴量選択 ===")
            analyzer.lasso_feature_selection()
            
            # Ridge回帰の実行
            print("\n=== Ridge回帰の実行 ===")
            analyzer.ridge_regression()
            
            # モデルの評価
            print("\n=== モデル評価 ===")
            results = analyzer.evaluate_models()
            
            for model_name, metrics in results.items():
                print(f"\n{model_name}:")
                print(f"  訓練R²: {metrics['train_r2']:.4f}")
                print(f"  テストR²: {metrics['test_r2']:.4f}")
                print(f"  訓練MSE: {metrics['train_mse']:.4f}")
                print(f"  テストMSE: {metrics['test_mse']:.4f}")
            
            # 結果の保存
            analyzer.save_results_summary(results)
            
            # 選択された特徴量の情報
            print("\n=== 選択された特徴量 ===")
            selected_info = analyzer.get_selected_features_info()
            if selected_info is not None:
                print(selected_info.head(10))
            
            # 可視化（結果も自動保存される）
            print("\n=== 可視化の作成 ===")
            analyzer.plot_coefficient_comparison()
            analyzer.plot_feature_selection_path()
            
            print(f"\n全ての結果が {output_dir} に保存されました")

if __name__ == "__main__":
    main()